# Ensembl downloader



<div><b>Getting species data from Ensembl</b><br></div>

In [2]:
import org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql.functions._

In [42]:
import better.files._
import File._
import java.io.{File => JFile}

In [3]:
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._

**Setting up release variable and folder to download ensembl**If the folders do no exist - creates them



In [56]:
import os.{GlobSyntax, /}
implicit val default_ammonite_permissions = os.PermSet(Integer.parseInt("777"))
default_ammonite_permissions 

r----x--x

In [41]:
val release = "101" //99 in the cross-species paper
val data = File("/data")
data.createDirectoryIfNotExists()
val ensembl_dir = data / "ensembl" / release
ensembl_dir.createDirectoryIfNotExists()

/data/ensembl/101

In [47]:
val indexes_dir = data / "indexes"
indexes_dir.createDirectoryIfNotExists()
val salmon_version = "1.3.0" //1.1.0 in the cross-species paper
val salmon_dir = indexes_dir / "salmon" 
salmon_dir.createDirectoryIfNotExists()
val salmon_indexes_dir =  salmon_dir / salmon_version
salmon_indexes_dir.createDirectoryIfNotExists()
val salmon_indexes_dir_str = salmon_indexes_dir.pathAsString
val salmon_ensembl_dir = salmon_indexes_dir / s"ensembl_${release}"
salmon_ensembl_dir.createDirectoryIfNotExists()
val salmon_ensembl_dir_str = salmon_ensembl_dir.pathAsString //to avoid serialization problems
salmon_ensembl_dir_str

/data/indexes/salmon/1.3.0/ensembl_101

In [48]:
val species_folder = ensembl_dir / "species"
species_folder.createDirectoryIfNotExists()
import ammonite.ops._    
val species_path_str = species_folder.pathAsString
species_path_str

/data/ensembl/101/species

In [58]:
val pipelines_dir = data / "pipelines"
pipelines_dir.createDirectoryIfNotExists()
val quantification_dir = pipelines_dir / "quantification"
quantification_dir.createDirectoryIfNotExists()
val quant_sample_dir = pipelines_dir / "quant_sample"
quant_sample_dir.createDirectoryIfNotExists()
val quant_by_runs_dir = pipelines_dir / "quant_by_runs"
quant_by_runs_dir.createDirectoryIfNotExists()
val pipelines_dir_str = pipelines_dir.pathAsString
pipelines_dir_str

/data/pipelines

Setting up the web client and URLs

In [43]:
import sttp.client3._
import sttp.model._
implicit val backend = HttpURLConnectionBackend()
val ensembl_ftp_str = s"ftp.ensembl.org/pub/release-${release}"
val ensembl_ftp = s"ftp://${ensembl_ftp_str}"
val ensembl_ftp_uri = uri"http://${ensembl_ftp_str}" //have to use http for requests :(
ensembl_ftp


ftp://ftp.ensembl.org/pub/release-101

In [46]:
val species_list = ensembl_dir / "species_EnsemblVertebrates.txt"
if(!species_list.exists){
    val species_url = uri"${ensembl_ftp}/species_EnsemblVertebrates.txt"
    val request = basicRequest.get(species_url)
    val Right(species_txt) = request.send(backend).body
    species_list.createFileIfNotExists()
    species_list.overwrite(species_txt)
}
species_list.pathAsString

/data/ensembl/101/species_EnsemblVertebrates.txt

In [4]:
val capitalize_first = udf[String, String] { (species) => species.head.toUpper + species.tail }
val species = spark.readTSV(species_list.pathAsString ,comment="", header=true)
.withColumnRenamed("#name", "name")
.withColumn("species", capitalize_first($"species"))
species.show()

+--------------------+--------------------+------------------+-----------+--------------------+------------------+--------------------+---------+-----------+---------------+-----------------+----------------+--------------------+----------+
|                name|             species|          division|taxonomy_id|            assembly|assembly_accession|           genebuild|variation|pan_compara|peptide_compara|genome_alignments|other_alignments|             core_db|species_id|
+--------------------+--------------------+------------------+-----------+--------------------+------------------+--------------------+---------+-----------+---------------+-----------------+----------------+--------------------+----------+
|       Spiny chromis|Acanthochromis_po...|EnsemblVertebrates|      80966|         ASM210954v1|   GCA_002109545.1|2018-05-Ensembl/2...|        N|          N|              Y|                Y|               Y|acanthochromis_po...|         1|
|Eurasian sparrowhawk|     Accipiter

#### Preparing to download genomes/transcriptomes/annotations from Ensembl




In [6]:
def prefix(prefix: String, sufix: String) =  udf[String, String]{ str=> prefix + str + sufix}

def command(path: String, ftp: String) = udf[String, String]{ species => {    
    val dir = path + "/" + species
    val wget = "wget -t 4 -m -nH --cut-dirs=100 -P " + dir + " "
    val and = " && "
    "mkdir -p " + dir + and + wget + ftp + "/gtf/" + species.toLowerCase + "/*" + and + wget + ftp + "/fasta/" + species.toLowerCase + "/*" + and + "gunzip -f " + dir + "/*.gz" 
    
}
}
val download = command(species_path_str, ensembl_ftp)


In [7]:
class Fixer{
    //fixes some typical naming problems
    import ammonite.ops._
   def no_dot(assembly: String) = { assembly.lastIndexOf(".") match { case -1 => assembly ; case i => assembly.substring(0, i) } }
 
    //def assembly2(prefix: String) = udf[String, String, String]{ (one, two) => prefix + one + "/" + two }
    def fix(str: String, assembly: String) = {
         if(exists! Path(str)) str else {
             val str2 = str.replace(assembly, no_dot(assembly))
             if(exists! Path(str2)) str2 else {
                 if(str.contains(" "))
                    str.replace(" ", "_")
                 else str
             }
         }
    }
}

In [8]:

def genome =  udf[String, String, String] { (species, assembly) => 
   val f = new Fixer(); import f._
    
    import ammonite.ops._    
    val species_path = Path(species_path_str)
    val head_tail_assembly = s"${species.head.toUpper + species.tail}.${assembly}"
    val p = (species_path / species / s"${head_tail_assembly}.dna.primary_assembly.fa".replace(" ", "")).toString
     
    //if(new java.io.File(p).exists) p else  if(exists! Path(fix(p, assembly))) fix(p, assembly) else {
    
    if(exists! Path(p)) p else  if(exists! Path(fix(p, assembly))) fix(p, assembly) else {
    val str = (species_path / species /  s"${head_tail_assembly}.dna.toplevel.fa".replace(" ", "")).toString 
    fix(str, assembly)
    }
}

def cdna = udf[String, String, String] { (species, assembly) => 
    val f = new Fixer(); import f._
    val species_path = Path(species_path_str)
    val head_tail_assembly = s"${species.head.toUpper + species.tail}.${assembly}"    
    val str = (species_path / species /  s"${head_tail_assembly}.cdna.all.fa".replace(" ", "")).toString
    fix(str, assembly)
}

def gtf = udf[String, String, String] { (species, assembly) => 
    val f = new Fixer(); import f._
    val species_path = Path(species_path_str)
    val head_tail_assembly = s"${species.head.toUpper + species.tail}.${assembly}"        
    val str =  (species_path / species /  s"${head_tail_assembly}.${release}.gtf".replace(" ", "")).toString
    fix(str, assembly)
}
def pep =  udf[String, String, String] { (species, assembly) => 
val f = new Fixer(); import f._
    val species_path = Path(species_path_str)
    val head_tail_assembly = s"${species.head.toUpper + species.tail}.${assembly}"        
    val str =  (species_path / species /   s"${head_tail_assembly}.pep.all.fa".replace(" ", "")).toString 
    fix(str, assembly)
}

def check_file_simple = udf[Boolean, String] { str =>
    new java.io.File(str).exists 
   // exists! ammonite.ops.Path(str)
}    

def new_index = udf[String, String] { 
    str => salmon_ensembl_dir_str +"/" + str.head.toUpper + str.tail }


<div><b>Setting up Salmon index</b></div><div>---------------------------------<br></div>

In [10]:
def gentrome( species: String, 
    genome: String, 
    transcriptome: String,
    version: String,
    subversion: String) = 
     s"""
     {
      "species": "${species}",
      "genome": "${genome}",
      "transcriptome": "${transcriptome}",
      "version": "${version}",
      "subversion": "${subversion}"
    }"""

def salmonIndex(
    species: String, 
    genome: String, 
    transcriptome: String,
    version: String,
    subversion: String,
    folder: String = ""    
    ) = s"""
{""" +
  (if(folder!="") s""" "quant_index.indexes_folder": "${folder}", """ else "") +
  s""" "quant_index.references": [
    ${gentrome(species, genome, transcriptome, version, subversion)}
  ]
}
""".replace("\t", "  ")
salmonIndex("Homo sapiens", s"/data/ensembl/${release}/species/Homo_sapiens/Homo_sapiens.GRCh38.dna.primary_assembly.fa", 
s"/data/ensembl/${release}/species/Homo_sapiens/Homo_sapiens.GRCh38.cdna.all.fa", "GRCh38", 
s"ensembl_${release}",s"/data/indexes/salmon/${salmon_version}/ensembl_${release}"
)


{ "quant_index.indexes_folder": "/data/indexes/salmon/1.3.0/ensembl_101",  "quant_index.references": [
    
     {
      "species": "Homo sapiens",
      "genome": "/data/ensembl/101/species/Homo_sapiens/Homo_sapiens.GRCh38.dna.primary_assembly.fa",
   

In [11]:
def index_input =  udf[String, String, String, String, String, String, String] {   (folder: String, species: String, genome: String, transcriptome: String, version: String, subversion: String) =>
  s"""
{
  "quant_index.indexes_folder": "${folder}",
  "quant_index.references": [
    {
      "species": "${species}",
      "genome": "${genome}",
      "transcriptome": "${transcriptome}",
      "version": "${version}",
      "subversion": "${subversion}"
    }
  ]
}
""".replace("\t", " ").replace("\n", "  ")
}

**Preparing the main table**




Computing genomes table




In [14]:
import org.apache.spark.sql.functions._
val genomes = species
  .withColumn("download", download($"species"))
  .withColumn("index", new_index($"species"))  
  .withColumn("genome", genome($"species", $"assembly"))
  .withColumn("cdna", cdna($"species", $"assembly"))
  .withColumn("index_input", index_input(lit(s"/data/indexes/salmon/${salmon_version}/ensembl_${release}"), $"species", $"genome", $"cdna", $"assembly", lit(s"ensembl_${release}")))
  .select("name", "species", "download", "assembly", "genome", "cdna", "index", "index_input") 
  .withColumn("index_exists", check_file_simple($"index"))
  .withColumn("genome_exists", check_file_simple($"genome"))
  .withColumn("cdna_exists", check_file_simple($"cdna"))
  .withColumn("gtf", gtf($"species", $"assembly"))
  .withColumn("gtf_exists", check_file_simple($"gtf"))
  .withColumn("pep", pep($"species", $"assembly"))
  .withColumn("pep_exists", check_file_simple($"pep"))
  .select("name", "index", "index_exists", "species", "download", "assembly", "genome", "genome_exists", "cdna", "cdna_exists", "gtf", "gtf_exists", "pep", "pep_exists")  
 genomes.show(100,1000)

+------------------------------+-----------------------------------------------------------------------+------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+--------------------------------------------------------------------------------------------------------------------------+-------------+----------------------------------------------------------------------------------------------------------------------+-----------+--------------------------------------------------------

In [15]:
genomes.where($"genome_exists" === false).show(1000, 10000)

+--------------------------------------+-----------------------------------------------------------------------+------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+--------------------------------------------------------------------------------------------------------------------------+-------------+----------------------------------------------------------------------------------------------------------------------+-----------+---------------------------------------------

In [17]:
genomes.select("index", "assembly", "index_exists").withColumn("index2", concat($"index" , lit("/") , $"assembly")).where($"index_exists" === false).show(100,1000)

+-----------------------------------------------------------------------+------------------------------+------------+------------------------------------------------------------------------------------------+
|                                                                  index|                      assembly|index_exists|                                                                                    index2|
+-----------------------------------------------------------------------+------------------------------+------------+------------------------------------------------------------------------------------------+
|     /data/indexes/salmon/1.3.0/ensembl_101/Acanthochromis_polyacanthus|                   ASM210954v1|       false|            /data/indexes/salmon/1.3.0/ensembl_101/Acanthochromis_polyacanthus/ASM210954v1|
|                 /data/indexes/salmon/1.3.0/ensembl_101/Accipiter_nisus|        Accipiter_nisus_ver1.0|       false|             /data/indexes/salmon/1.3.0/ensembl

**Computing table of files to download**




In [20]:
import ammonite.ops._
val to_download = genomes.where($"cdna_exists" === false).select("download").as[String] 
val already_downloaded = genomes.where($"cdna_exists" === true).select("download").as[String] 
println("ALREADY DOWNLOADED: "+already_downloaded.count)
println("TO DOWNLOAD: "+to_download.count)
genomes.where($"genome_exists" === false).select("species").sort($"species".asc).show(1000)

ALREADY DOWNLOADED: 1
TO DOWNLOAD: 309
+--------------------+
|             species|
+--------------------+
|Acanthochromis_po...|
|     Accipiter_nisus|
|Ailuropoda_melano...|
|    Amazona_collaria|
|Amphilophus_citri...|
|Amphiprion_ocellaris|
|  Amphiprion_percula|
|  Anabas_testudineus|
|  Anas_platyrhynchos|
|Anas_platyrhyncho...|
|    Anas_zonorhyncha|
| Anolis_carolinensis|
|Anser_brachyrhynchus|
|     Anser_cygnoides|
|     Aotus_nancymaae|
|     Apteryx_haastii|
|      Apteryx_owenii|
|        Apteryx_rowi|
|Aquila_chrysaetos...|
|Astatotilapia_cal...|
|  Astyanax_mexicanus|
|Astyanax_mexicanu...|
|  Athene_cunicularia|
|Balaenoptera_musc...|
|     Betta_splendens|
|   Bison_bison_bison|
|       Bos_grunniens|
|  Bos_indicus_hybrid|
|           Bos_mutus|
|          Bos_taurus|
|   Bos_taurus_hybrid|
|           Bubo_bubo|
|     Buteo_japonicus|
|Caenorhabditis_el...|
|Cairina_moschata_...|
|     Calidris_pugnax|
|    Calidris_pygmaea|
|  Callithrix_jacchus|
| Callorhinchus_mi

In [50]:
//val to_download = genomes.where($"species" === "Homo_sapiens").select("download").as[String] 
val str = to_download.as[String].collect.toList.mkString("\n")
//str

In [21]:
write.over(Path( (ensembl_dir / "selected_download.sh").pathAsString), str, perms = default_ammonite_permissions)

# NOTE: After this step we assume that the user started selected_download.sh script in bash and waited until download finished

What youn can also do: once again check which species were downloaded and have indexes



In [54]:
genomes.select("index", "assembly", "index_exists").withColumn("index2", concat($"index" , lit("/") , $"assembly")).where($"index_exists" === false).show(100,1000)

+-----------------------------------------------------------------------+------------------------------+------------+------------------------------------------------------------------------------------------+
|                                                                  index|                      assembly|index_exists|                                                                                    index2|
+-----------------------------------------------------------------------+------------------------------+------------+------------------------------------------------------------------------------------------+
|     /data/indexes/salmon/1.3.0/ensembl_101/Acanthochromis_polyacanthus|                   ASM210954v1|       false|            /data/indexes/salmon/1.3.0/ensembl_101/Acanthochromis_polyacanthus/ASM210954v1|
|                 /data/indexes/salmon/1.3.0/ensembl_101/Accipiter_nisus|        Accipiter_nisus_ver1.0|       false|             /data/indexes/salmon/1.3.0/ensembl

In [55]:
import ammonite.ops._
val to_download = genomes.where($"cdna_exists" === false).select("download").as[String] 
val already_downloaded = genomes.where($"cdna_exists" === true).select("download").as[String] 
println("ALREADY DOWNLOADED: "+already_downloaded.count)
println("TO DOWNLOAD: "+to_download.count)
genomes.where($"genome_exists" === false).select("species").sort($"species".asc).show(1000)

ALREADY DOWNLOADED: 1
TO DOWNLOAD: 309
+--------------------+
|             species|
+--------------------+
|Acanthochromis_po...|
|     Accipiter_nisus|
|Ailuropoda_melano...|
|    Amazona_collaria|
|Amphilophus_citri...|
|Amphiprion_ocellaris|
|  Amphiprion_percula|
|  Anabas_testudineus|
|  Anas_platyrhynchos|
|Anas_platyrhyncho...|
|    Anas_zonorhyncha|
| Anolis_carolinensis|
|Anser_brachyrhynchus|
|     Anser_cygnoides|
|     Aotus_nancymaae|
|     Apteryx_haastii|
|      Apteryx_owenii|
|        Apteryx_rowi|
|Aquila_chrysaetos...|
|Astatotilapia_cal...|
|  Astyanax_mexicanus|
|Astyanax_mexicanu...|
|  Athene_cunicularia|
|Balaenoptera_musc...|
|     Betta_splendens|
|   Bison_bison_bison|
|       Bos_grunniens|
|  Bos_indicus_hybrid|
|           Bos_mutus|
|          Bos_taurus|
|   Bos_taurus_hybrid|
|           Bubo_bubo|
|     Buteo_japonicus|
|Caenorhabditis_el...|
|Cairina_moschata_...|
|     Calidris_pugnax|
|    Calidris_pygmaea|
|  Callithrix_jacchus|
| Callorhinchus_mi

### Writing json-s for salmon indexes

----------------------------------------------------------------

Starting from checking which indexes are already build and then preparing jsons to build the missing ones




In [23]:
val already_indexed = genomes.where($"index_exists" === true)
val to_index = genomes.where($"index_exists" === false)
println("ALREADY INDEXED: "+already_indexed.count)
println("TO INDEX: "+to_index.count)


ALREADY INDEXED: 0
TO INDEX: 310


In [24]:
val download_data =  species
  .withColumn("download", download($"species"))
  .withColumn("index", new_index($"species"))
  .withColumn("genome", genome($"species", $"assembly"))
  .withColumn("cdna", cdna($"species", $"assembly"))
  .withColumn("index", concat($"index" , lit("/") , $"assembly"))
  .select("name", "species", "download", "assembly", "genome", "cdna") 
  .as[(String, String, String, String, String , String)].collect.toList
  download_data.head

(Spiny chromis,Acanthochromis_polyacanthus,mkdir -p /data/ensembl/101/species/Acanthochromis_polyacanthus && wget -t 4 -m -nH --cut-dirs=100 -P /data/ensembl/101/species/Acanthochromis_polyacanthus ftp://ftp.ensembl.org/pub/release-101/gtf/acanthochromis

Salmon indexes inputs
-----------------------

In [26]:
val salmon_inputs_dir = ensembl_dir / "inputs" / "salmon" 
val by_name = download_data.map{ case (name, species, download, assembly, genome, cdna) => 
(salmon_inputs_dir / "by_name" / s"${name.replace("/","-")}" / s"${species}_${assembly}.json").pathAsString.replace(" ", "_") ->  
salmonIndex(species, genome, cdna, assembly, s"ensembl_${release}", salmon_ensembl_dir_str)  }.toMap
by_name.head._2



{ "quant_index.indexes_folder": "/data/indexes/salmon/1.3.0/ensembl_101",  "quant_index.references": [
    
     {
      "species": "Taeniopygia_guttata",
      "genome": "/data/ensembl/101/species/Taeniopygia_guttata/Taeniopygia_guttata.bTaeGut1_v1.p.d

In [57]:
val by_species = download_data.map{ 
    case (name, species, download, assembly, genome, cdna) => 
    (salmon_inputs_dir / "by_species" / s"${species}_${assembly}.json").pathAsString.replace(" ", "_") ->  
    salmonIndex(species, genome, cdna, assembly, s"ensembl_${release}", salmon_ensembl_dir_str) 
    }.toMap
by_species.head

(/data/ensembl/101/inputs/salmon/by_species/Amphiprion_ocellaris_AmpOce1.0.json,
{ "quant_index.indexes_folder": "/data/indexes/salmon/1.3.0/ensembl_101",  "quant_index.references": [
    
     {
      "species": "Amphiprion_ocellaris",
      "genome": "

In [28]:
import os.{GlobSyntax, /}
import ammonite.ops._
for{(d, i) <- by_species } write.over(Path(d), i, createFolders = true, perms = default_ammonite_permissions)
for{(d, i) <- by_name } write.over(Path(d), i, createFolders = true, perms = default_ammonite_permissions)


<div><b>Salmon indexes batches</b></div><div>---------------------------------<br></div><div>Created indexes batches to build indexes for several species together</div>

In [31]:
val sl = 3
val batches = download_data.map{ case (name, species, download, assembly, genome, cdna) => 
    gentrome(species, genome, cdna, assembly, s"ensembl_${release}") 
}.sliding(sl, sl).map{case b=>
val str = b.mkString("[", ",", "]")

s"""
{
  "quant_index_batch.indexes_folder": ${salmon_indexes_dir_str},
  "quant_index_batch.threads_per_index": ${32 / sl},
  "quant_index_batch.references": ${str}",
  "quant_index_batch.memory_per_index": "24G"
}
"""
}.toList
batches.head



{
  "quant_index_batch.indexes_folder": /data/indexes/salmon/1.3.0,
  "quant_index_batch.threads_per_index": 10,
  "quant_index_batch.references": [
     {
      "species": "Acanthochromis_polyacanthus",
      "genome": "/data/ensembl/101/species/Acanth

## Preparing default inputs

Writes default input.json for the pipelines




In [60]:
val samples_dir = data / "samples"
val samples_dir_str = samples_dir.pathAsString
samples_dir_str

/data/samples

In [34]:
import spark.implicits._
val gs = genomes.withColumn("index", concat($"index", lit("/"), $"assembly", lit(s"_ensembl_${release}"))).select("species", "index", "gtf").as[(String, String, String)].collect().toList

In [35]:
val salmon_indexes_str = gs.collect{
    case (sp, i, _) => "\"" + sp.head.toUpper + sp.tail.replace("_", " ") +  "\"" +" : " +  "\"" + i  + "\""
}.mkString(",\n")
val salmon_gtfs_str = gs.collect{
    case (sp, _, gtf) => "\"" + sp.head.toUpper + sp.tail.replace("_", " ") +  "\"" +" : " +  "\"" + gtf  + "\""
}.mkString(",\n")
//salmon_gtfs_str

In [59]:
val key = "0a1d74f32382b8a154acacc3a024bdce3709"

In [36]:
val quant_sample_default = s"""
    {
  "quant_sample.key": $key,
  "quant_sample.samples_folder": $samples_dir_str,
  "quant_sample.salmon_indexes" : {
    ${salmon_indexes_str}   
  },
  "quant_sample.transcripts2genes" : {
     ${salmon_gtfs_str}
  }
}
"""
write.over(Path((quant_sample_dir / "quant_sample_default.json").pathAsString), quant_sample_default, createFolders = true, perms = default_ammonite_permissions)

In [37]:
val quant_by_runs_default = s"""
    {
  "quant_by_runs.key": $key,
  "quant_by_runs.samples_folder": $samples_dir_str,
  "quant_by_runs.salmon_indexes" : {
    ${salmon_indexes_str}   
  },
  "quant_by_runs.transcripts2genes" : {
     ${salmon_gtfs_str}
  }
}
"""
write.over(Path((quant_by_runs_dir / "quant_by_runs_default.json").pathAsString), 
quant_by_runs_default, createFolders = true, perms = default_ammonite_permissions)

In [38]:
val quantification_def = s"""
    {
  "quantification.key": $key,
  "quantification.samples_folder": "/data/samples/species",
  "quantification.salmon_indexes" : {
    ${salmon_indexes_str}   
  },
  "quantification.transcripts2genes" : {
     ${salmon_gtfs_str}
  }
}
"""
write.over(Path((quantification_dir / "quantification_default.json").pathAsString), quantification_def,  perms = default_ammonite_permissions, createFolders = true)